In [9]:
#!pip install langchain langchain-google-genai "unstructured[pdf]" duckdb langchain-community chromadb pandas

In [10]:
import duckdb
import pandas as pd
from datetime import datetime

#### 1. Processing complex PDF with Unstructured

In [11]:
from langchain_community.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader("../../data/relatorio_vendas.pdf", mode="elements")
docs_unstructured = loader.load()

print(f"Total elements: {len(docs_unstructured)}")

for doc in docs_unstructured:
    print(f"- Element type: {doc.metadata.get('category')}")
    print(f"{doc.page_content}\n")

Total elements: 23
- Element type: NarrativeText
Relatório Trimestral de Vendas - Q1 2024 Este relatório apresenta uma análise detalhada das vendas no primeiro trimestre de 2024. A performance geral foi positiva, com crescimento em todas as categorias de produtos. A seguir, uma tabela com os resultados por produto.

- Element type: Title
ID Produto

- Element type: Title
Nome do Produto

- Element type: Title
Categoria Unidades Vendidas Receita (R$)

- Element type: Title
PROD-001

- Element type: Title
Laptop Pro X

- Element type: Title
Eletrônicos

- Element type: UncategorizedText
1500

- Element type: UncategorizedText
7.500.000

- Element type: Title
PROD-002 Cadeira Ergonômica Mobiliário

- Element type: UncategorizedText
2500

- Element type: UncategorizedText
1.250.000

- Element type: Title
PROD-003

- Element type: Title
Software de Análise

- Element type: Title
Software

- Element type: UncategorizedText
500

- Element type: UncategorizedText
2.000.000

- Element type: Tit

##### Adding strategic metadata

In [12]:
from langchain_core.documents import Document

docs_com_metadados = []

for doc in docs_unstructured:
    novos_metadados = doc.metadata.copy()
    novos_metadados["source"] = "../../data/relatorio_vendas.pdf"
    novos_metadados["ingestion_date"] = datetime.now().strftime("%Y-%m-%d")
    novos_metadados["data_owner"] = "Sales Department"

    docs_com_metadados.append(
        Document(page_content=doc.page_content, metadata=novos_metadados)
    )

print(f"Total elements: {len(docs_com_metadados)}\n")

print(f"Original:              {docs_unstructured[-1]}\n")
print(f"Metadados added: {docs_com_metadados[-1]}")


Total elements: 23

Original:              page_content='A categoria de Eletrônicos continua a ser a mais lucrativa. A estratégia para o Q2 será focar em marketing para a Cadeira Ergonômica, que possui alto volume de vendas mas menor receita.' metadata={'source': '../../data/relatorio_vendas.pdf', 'coordinates': {'points': ((78.0, 291.07), (78.0, 313.07), (502.6299999999997, 313.07), (502.6299999999997, 291.07)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}, 'file_directory': '../../data', 'filename': 'relatorio_vendas.pdf', 'last_modified': '2025-12-25T16:23:16', 'page_number': 1, 'languages': ['por'], 'filetype': 'application/pdf', 'parent_id': '3fdda247a3f19f65b3ae66bc22fd1944', 'category': 'NarrativeText', 'element_id': '70b91eb7a59a15529f34b3b93f6d9ff0'}

Metadados added: page_content='A categoria de Eletrônicos continua a ser a mais lucrativa. A estratégia para o Q2 será focar em marketing para a Cadeira Ergonômica, que possui alto volume de vendas mas m

#### 2. Smart chunking with RecursiveCharacterTextSplitter

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = text_splitter.split_documents(docs_com_metadados)

print(f"Total documents before chunking: {len(docs_com_metadados)}")
print(f"Total chunks: {len(chunks)}")

print(chunks[2])

Total documents before chunking: 23
Total chunks: 23
page_content='Nome do Produto' metadata={'source': '../../data/relatorio_vendas.pdf', 'coordinates': {'points': ((177.86, 171.07000000000005), (177.86, 181.07000000000005), (261.75, 181.07000000000005), (261.75, 171.07000000000005)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}, 'file_directory': '../../data', 'filename': 'relatorio_vendas.pdf', 'last_modified': '2025-12-25T16:23:16', 'page_number': 1, 'languages': ['por'], 'filetype': 'application/pdf', 'category': 'Title', 'element_id': '8db19212a1dd1d7912af3dad01463352', 'ingestion_date': '2026-01-09', 'data_owner': 'Sales Department'}


#### 3. Data ingestion with DuckDB

In [14]:
import duckdb
import pandas as pd

In [15]:
# Conecting/Creating DB
con = duckdb.connect(database=':memory:', read_only=False)

# Creating table
con.execute("""
CREATE TABLE produtos (
    id INTEGER PRIMARY KEY,
    nome VARCHAR,
    categoria VARCHAR,
    preco FLOAT,
    estoque INTEGER,
    descricao VARCHAR);""")

# Inserting example data
produtos_df = pd.DataFrame({
    'id': [101, 102, 103, 104],
    'nome': ['Laptop Gamer Z', 'Mouse Óptico Fast', 'Teclado Mecânico Pro', 'Monitor Curvo 34"'],
    'categoria': ['Eletrônicos', 'Acessórios', 'Acessórios', 'Eletrônicos'],
    'preco': [9500.00, 250.00, 800.00, 3200.00],
    'estoque': [15, 120, 60, 25],
    'descricao': [
        'Laptop de alta performance com placa de vídeo dedicada e 32GB RAM.',
        'Mouse com 16.000 DPI e design ergonômico para longas sessões.',
        'Teclado com switches mecânicos, RGB e layout ABNT2.',
        'Monitor ultrawide com alta taxa de atualização e cores vibrantes.'
    ]
})
con.register("produtos_df", produtos_df)
con.execute("INSERT INTO produtos SELECT * FROM produtos_df")

# Verifying data
print(con.execute("SELECT * FROM produtos").fetchdf())

    id                  nome    categoria   preco  estoque  \
0  101        Laptop Gamer Z  Eletrônicos  9500.0       15   
1  102     Mouse Óptico Fast   Acessórios   250.0      120   
2  103  Teclado Mecânico Pro   Acessórios   800.0       60   
3  104     Monitor Curvo 34"  Eletrônicos  3200.0       25   

                                           descricao  
0  Laptop de alta performance com placa de vídeo ...  
1  Mouse com 16.000 DPI e design ergonômico para ...  
2  Teclado com switches mecânicos, RGB e layout A...  
3  Monitor ultrawide com alta taxa de atualização...  


##### Transforming SQL lines into Documents

In [16]:
df_produtos = con.execute("SELECT * FROM produtos").fetchdf()

docs_sql = []
for _, row in df_produtos.iterrows():
    page_content = f"Produto: {row['nome']}. Categoria: {row['categoria']}. Preço: R$ {row['preco']:.2f}. Estoque: {row['estoque']}. Descrição: {row['descricao']}."

    metadata = {
        "source": "tabela_produtos_duckdb",
        "produto_id": row['id'],
        "categoria": row['categoria'],
        "preco": row['preco'],
        "ingestion_date": datetime.now().strftime("%Y-%m-%d")
    }

    docs_sql.append(
        Document(page_content=page_content, metadata=metadata)
    )

con.close()

print(f"Total documents from SQL: {len(docs_sql)}\n")
print(f"Document example from DB:\n{docs_sql[0]}")

Total documents from SQL: 4

Document example from DB:
page_content='Produto: Laptop Gamer Z. Categoria: Eletrônicos. Preço: R$ 9500.00. Estoque: 15. Descrição: Laptop de alta performance com placa de vídeo dedicada e 32GB RAM..' metadata={'source': 'tabela_produtos_duckdb', 'produto_id': 101, 'categoria': 'Eletrônicos', 'preco': 9500.0, 'ingestion_date': '2026-01-09'}


#### 4. Uniting pipelines and sending to Vector Store

In [17]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata

final_docs = chunks + docs_sql
print(f"Total documents to be indexed: {len(final_docs)}\n")

filtered_docs = filter_complex_metadata(final_docs)
print(f"Total documents after filtering: {len(filtered_docs)}\n")

embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

vector_store = Chroma.from_documents(
    documents=filtered_docs,
    embedding=embeddings
)

Total documents to be indexed: 27

Total documents after filtering: 27



In [18]:
pdf_question = "Qual foi a receita com laptop?"
pdf_result = vector_store.similarity_search(pdf_question, k=2)
print(f"Pergunta: {pdf_question}")
for doc in pdf_result:
    print(f"- Similaridade: {doc.page_content}")
    print(f" (Fonte: {doc.metadata.get("source")}, Categoria:{doc.metadata.get("categoria")})\n")

print("-"*20)

sql_question = "Me fale sobre o teclado mecânico"
sql_result = vector_store.similarity_search(sql_question, k=2)
print(f"Pergunta: {sql_question}")
for doc in sql_result:
    print(f"- Similaridade: {doc.page_content}")
    print(f" (Fonte: {doc.metadata.get("source")}, Categoria:{doc.metadata.get("categoria")})\n")


Pergunta: Qual foi a receita com laptop?
- Similaridade: Laptop Pro X
 (Fonte: ../../data/relatorio_vendas.pdf, Categoria:None)

- Similaridade: Produto: Laptop Gamer Z. Categoria: Eletrônicos. Preço: R$ 9500.00. Estoque: 15. Descrição: Laptop de alta performance com placa de vídeo dedicada e 32GB RAM..
 (Fonte: tabela_produtos_duckdb, Categoria:Eletrônicos)

--------------------
Pergunta: Me fale sobre o teclado mecânico
- Similaridade: Produto: Teclado Mecânico Pro. Categoria: Acessórios. Preço: R$ 800.00. Estoque: 60. Descrição: Teclado com switches mecânicos, RGB e layout ABNT2..
 (Fonte: tabela_produtos_duckdb, Categoria:Acessórios)

- Similaridade: Eletrônicos
 (Fonte: ../../data/relatorio_vendas.pdf, Categoria:None)



## 📚 Resumo Prático da Aula 4

- **Use a ferramenta certa**: `UnstructuredPDFLoader` é superior ao `PyPDFLoader` para documentos com estruturas complexas como tabelas.
- **Metadados são seu melhor amigo**: Enriquecer os documentos durante a ingestão com informações de fonte, datas e outros atributos é o que permite criar RAGs realmente úteis e confiáveis.
- **RAG não é só para texto**: Transformar dados estruturados (de SQL, CSVs, etc.) em documentos textuais expande enormemente o conhecimento do seu sistema.
- **Pipeline é um processo**: O fluxo `Load -> Transform (Add Metadata) -> Split -> Index` é um padrão robusto para a maioria das necessidades de ingestão de dados em RAG.